In [14]:
from vega_datasets import data
import altair as alt

In [20]:
df = data.iris()
df.columns[0:-1]

Index(['sepalLength', 'sepalWidth', 'petalLength', 'petalWidth'], dtype='object')

In [55]:
alt.Chart(df).mark_point().encode(
    y = 'species',
    x=alt.X(alt.repeat(), type='quantitative', scale=alt.Scale(zero=False)),
).repeat(repeat=list(df.columns[0:-1]))

alt.RepeatChart(...)

,sepalLength,sepalWidth,petalLength,petalWidth,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5.0,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa
9,4.9,3.1,1.5,0.1,setosa


In [ ]:
for col in features: 
    
    chart = 

In [186]:
# (alt.Chart(movies)
#  .transform_density(
#     'runtime',
#     as_=['runtime', 'density'])  # Give a name to the KDE values, which we can use when plotting
#  .mark_area(interpolate='monotone').encode(
#     x='runtime',
#     y='density:Q'))
single = alt.selection_single()



chart1 = alt.Chart(df).transform_density(
    'sepalLength',
    as_=['sepalLength', 'density'],
    groupby=['species']
    ).mark_area(interpolate='monotone', opacity=0.7).encode(
    y = 'density:Q',
    x = alt.X('sepalLength'), #alt.X(alt.repeat(), type='quantitative', scale=alt.Scale(zero=False)),
    color='species'
) 


features = ['sepalLength','sepalWidth', 'petalWidth']
chartlist = []

for feat in features:
    chart = alt.Chart(df).transform_density(
        feat,
        as_=[feat, 'density'],
        groupby=[]
        ).mark_area(interpolate='monotone', opacity=0.7).encode(
        y = 'density:Q',
        x = alt.X(feat), #alt.X(alt.repeat(), type='quantitative', scale=alt.Scale(zero=False)),
        color='species'
    ) 
    chartlist.append(chart)

for i in range(len(chartlist)):  
    print(i)
    if i == 0:
        output = chartlist[i]
    else:
        output = alt.vconcat(output, chartlist[i])

output
        

0
1
2


alt.VConcatChart(...)

In [242]:
def arid_eda(data_frame, response, features=[]):
    """
    
    Function to create summary statistics and basic EDA plots. Given a data frame,
    this function outputs general exploratory analysis plots as well as basic 
    statistics summarizing trends in the features of the input data. 
    
    Parameters
    ----------
    data_frame : pandas.DataFrame
        The input dataframe to analyze
    response : str
        A column name of the response variable
    features : list
        A list of the feature names to perform EDA on
    
    Returns
    -------
    altair.Chart
        Plots relevant to the exploratory data analysis
    
    pandas.DataFrame
        A dataframe containing summary statistics relevant to the 
        selected feature and response variable.
        
    Examples
    --------
    >>> from aridanalysis import aridanalysis
    >>> arid_eda(house_prices, 'price', ['rooms', 'age','garage'])
    """
    chartlist = []
    plot_width = 70*len(features)
    plot_height = 70*len(features)
    
    
    if type(response) is str:           # 
        group = [response]
    else: 
        group = []

    for feat in features:                            ### This function creates density plots for each feature 
        chart = alt.Chart(df).transform_density(     ### only works currently if response is categorical 
            feat,
            as_=[feat, 'density'],
            groupby=group
            ).mark_area(interpolate='monotone', opacity=0.7).encode(
            y = 'density:Q',
            x = alt.X(feat),
            color=response
        ) 
        chartlist.append(chart)

    for i in range(len(chartlist)):  
        if i == 0:
            dist_output = chartlist[i]
        else:
            dist_output = alt.vconcat(dist_output, chartlist[i])

    feature_df = df.loc[:,features]
    corr_df = feature_df.corr('spearman').stack().reset_index(name='corr')
    corr_df.loc[corr_df['corr'] == 1, 'corr'] = 0
    corr_df['abs'] = corr_df['corr'].abs()
    corr_plot = alt.Chart(corr_df).mark_rect().encode(
        x='level_0',
        y='level_1',
        size='abs',
        color=alt.Color('corr', scale=alt.Scale(scheme='blueorange'))
    ).properties(width=plot_width, height=plot_height)

    return dist_output | corr_plot

In [243]:
arid_eda(df, 'species', features = ['sepalLength','sepalWidth', 'petalWidth'])

alt.HConcatChart(...)

In [216]:
features

['sepalLength', 'sepalWidth', 'petalWidth']

In [233]:
plot_width = 70*len(features)
plot_height = 70*len(features)

feature_df = df.loc[:,features]
corr_df = feature_df.corr('spearman').stack().reset_index(name='corr')
corr_df.loc[corr_df['corr'] == 1, 'corr'] = 0
corr_df['abs'] = corr_df['corr'].abs()
corr_plot = alt.Chart(corr_df).mark_rect().encode(
    x='level_0',
    y='level_1',
    size='abs',
    color=alt.Color('corr', scale=alt.Scale(scheme='blueorange'))
).properties(width=plot_width, height=plot_height)

corr_plot

alt.Chart(...)

,level_0,level_1,corr
0,sepalLength,sepalLength,0.000000
1,sepalLength,sepalWidth,-0.166778
2,sepalLength,petalWidth,0.834289
3,sepalWidth,sepalLength,-0.166778
4,sepalWidth,sepalWidth,0.000000
5,sepalWidth,petalWidth,-0.289032
6,petalWidth,sepalLength,0.834289
7,petalWidth,sepalWidth,-0.289032
8,petalWidth,petalWidth,0.000000


In [ ]:
corr_df = movies.select_dtypes('number').corr('spearman').stack().reset_index(name='corr')
corr_df.loc[corr_df['corr'] == 1, 'corr'] = 0
corr_df['abs'] = corr_df['corr'].abs()
alt.Chart(corr_df).mark_circle().encode(
    x='level_0',
    y='level_1',
    size='abs',
    color=alt.Color('corr', scale=alt.Scale(scheme='blueorange')))

In [124]:
single = alt.selection_single()

chart1 = alt.Chart(df).mark_area().encode(
    y = 'count()',
    x = alt.X('sepalLength', bin=alt.Bin())#alt.X(alt.repeat(), type='quantitative', scale=alt.Scale(zero=False)),
    ,fill=alt.condition(single, 'species', alt.value('lightgray'))
).add_selection(single)

chart2 = alt.Chart(df.query('species=="setosa"')).mark_bar().encode(
    y = 'count()',
    x = alt.X('sepalLength', bin=alt.Bin())#alt.X(alt.repeat(), type='quantitative', scale=alt.Scale(zero=False)),
)
chart3 = alt.Chart(df.query('species=="versicolor"')).mark_bar().encode(
    y = 'count()',
    x = alt.X('sepalLength', bin=alt.Bin())#alt.X(alt.repeat(), type='quantitative', scale=alt.Scale(zero=False)),
)


In [125]:
chart1


alt.Chart(...)

In [100]:


chart2

alt.Chart(...)

alt.HConcatChart(...)